In [41]:
import numpy as np
import pandas as pd
from scipy.stats import rv_continuous, rv_discrete
from sklearn.preprocessing import StandardScaler
import pdb


classification_equations = []
classification_equations.append(dict())



def make_classification(data, data_long, predictive_features = ['bmi_slope']):
    """return a classification"""
    labmaxes = []
    labcounts = []
    bmislopes = []
    patient_long_features = []
#     pdb.set_trace()
    for patient in data: 
        # pdb.set_trace()
        ld = [d for d in data_long if d['id'] == patient['id']]
        patient_long = {} 
        patient_feature ={} 
        patient_long['id'] = patient['id']
        patient_feature['id'] = patient['id']
        bmis = []
        dates = []
        labmax = 0
        gcount=0
        for d in ld:
            if d['name'] not in patient_long:
                patient_long[d['name']] = {'value':[],'date':[]}
            patient_long[d['name']]['value'].append(d['value'])
            patient_long[d['name']]['date'].append(d['date'])
        keys = [ k for k in patient_long.keys() if k != 'id']
#         print('keys:',keys)
#         for k,v in patient_long.items():
#             print(k,v)
#         pdb.set_trace()
        for k in keys:
#             print(k,patient_long[k])
            patient_feature[k+'_max'] = np.max(patient_long[k]['value'])
            patient_feature[k+'_min'] = np.min(patient_long[k]['value'])
            patient_feature[k+'_median'] = np.median(patient_long[k]['value'])
            patient_feature[k+'_mean'] = np.mean(patient_long[k]['value'])
            patient_feature[k+'_var'] = np.var(patient_long[k]['value'])
            patient_feature[k+'_slope'] = (np.cov(patient_long[k]['value'], 
                                                  patient_long[k]['date'],
                                             ddof=0)[0,1] 
                                        / np.var(patient_long[k]['date']))
            patient_feature[k+'_count'] = float(len(patient_long[k]['value']))
        # date
#         patient_feature['date'] = patient_long['bmi'][1]
#         print('patient feature:',patient_feature)    
        # append patient to df
        patient_long_features.append(patient_feature)
        if np.var(dates) < 0.0001:
            pdb.set_trace()
#         pdb.set_trace()
    features = pd.DataFrame.from_records(patient_long_features).set_index('id')
#     for col in [c for c in features.columns if 'count' not in c]:
#         features[col] -= features[col].mean()
#         features[col] /= features[col].std()
    features -= features.mean()
    features /= features.std()

#         labmaxes.append({'id':patient['id'],'value':labmax})
#         labcounts.append(gcount)
#         bmislopes.append({'id':patient['id'],
#                           'bmislopes':bmislope})
   
#     df = pd.DataFrame.from_records(bmislopes).set_index('id')
#     df['bmislopes'] -= df['bmislopes'].mean()
#     df['bmislopes'] /= df['bmislopes'].std()
    
#     labmaxes = (np.array(labmaxes) - np.min(labmaxes)) / (np.max(labmaxes) -np.min(labmaxes))    
    BX = 0 
    BX_eqn = ''
    for i,pf in enumerate(predictive_features):
#         weight = np.random.randint(-2,2) + np.random.rand()
#         weight = np.random.randint(-1,1)
        weight = 1.0
        BX += weight*features[pf] #+ features['lab_max']
        if i != 0 : BX_eqn += '+'
        BX_eqn += str(round(weight,3)) +"*"+pf
    print('BX = ',BX_eqn)
    
    if (not classification_equations[0]):
        classification_equations[0] = dict()
        
    classification_equations[0][str("-".join(predictive_features))] = BX_eqn
    
    
    features['classification'] = (np.exp(BX)/(1+np.exp(BX)) > 0.5)
#     pdb.set_trace()
#     print('class:',features['classification'])
    return features 
    # return 1 if bmislope > 0.0 and labmax > 0.8 else 0



In [42]:
np.random.seed(42)
# generate data
ptid = np.arange(1000)
measurements=['bmi','age','lab']
data=[]
data_long=[]

for p in ptid:
    # tabular data
    pdict = {}
    pdict['id']=p
    pdict['sex']=np.random.randint(0,2)
#     pdict['sex']=0#np.random.randint(0,2)
    pdict['race']=np.random.randint(0,6)
#     pdict['race']=1 #np.random.randint(0,6)
    data.append(pdict)
    # long data 
    age = np.random.randint(18,85)
    date = np.random.randint(1000,5000)
    last_bmi = 30 + 5*np.random.randn()
    last_lab = np.random.randn()
    
    for visit in np.arange(np.random.randint(2,10)):

        age = age + np.random.randint(1,4) 
        date = date + np.random.randint(365,3*365) 

        for m in measurements:
            plongdict={}
            plongdict['id'] = p
            plongdict['name']=m
            plongdict['date']=date 

            if m == 'bmi':
                if visit == 0:
                    plongdict['value'] = last_bmi
                else:
                    plongdict['value'] = last_bmi + 2*np.random.randn() 
                    last_bmi = plongdict['value']
            elif m == 'age':
                plongdict['value'] = age
            elif m == 'lab': 
                if visit == 0:
                    plongdict['value'] = last_lab
                else:
                    plongdict['value'] = last_lab + np.random.randn() 
                    last_lab = plongdict['value']

            data_long.append(plongdict)


predictive_features = [ ['bmi_slope'],
                        ['lab_max'],
                        ['lab_min'],
                        ['lab_var'],
                        ['bmi_slope','lab_max'],
                        ['bmi_slope','lab_slope'],
                        ['bmi_slope','lab_var'],
                        ['bmi_slope','lab_median','age_max'],
                      ]
for pf in predictive_features:
    print('pf:',pf)
    df_outcome = make_classification(data, data_long, 
                                     pf) 
    # print('df_outcome:',df_outcome)
    for i,d in enumerate(data): 
        d['classification'] = df_outcome.loc[df_outcome.index==d['id']]['classification'].values[0]
    #     d['bmi_slope'] = df_outcome.loc[df_outcome.index==d['id']]['bmi_slope'].values[0]
    #     for col in df_outcome.loc[df_outcome.index==d['id']].columns:
    #         d[col] = df_outcome.loc[d['id']]['bmi_slope']
    #         d[col] = df_outcome.loc[d['id']]['lab_max']
    df = pd.DataFrame.from_records(data, index='id')
    print('df columns:',df.columns)
    df.sort_index(axis=0,inplace=True)   
    df.to_csv('data/d_lr_example_patients_'+'-'.join(pf)+'.csv')
   ## df.to_csv('d_lr_example_patients_'+'-'.join(pf)+'.csv')
    df_long = pd.DataFrame.from_records(data_long, index='id', columns=['id','name','date','value'])
    print(np.sum(df['classification'].values==True),'controls,',np.sum(df['classification'].values==False),'cases')
    #shuffle rows
    df_long.to_csv('data/d_lr_example_patients_'+'-'.join(pf)+'_long.csv')
   ## df_long.to_csv('d_lr_example_patients_'+'-'.join(pf)+'_long.csv')
 #####   plot_long(df_outcome,df,pf)

pf: ['bmi_slope']


/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


BX =  1.0*bmi_slope
df columns: Index(['classification', 'race', 'sex'], dtype='object')
499 controls, 501 cases
pf: ['lab_max']


/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


BX =  1.0*lab_max
df columns: Index(['classification', 'race', 'sex'], dtype='object')
459 controls, 541 cases
pf: ['lab_min']


...

BX =  1.0*bmi_slope+1.0*lab_slope
df columns: Index(['classification', 'race', 'sex'], dtype='object')
493 controls, 507 cases
pf: ['bmi_slope', 'lab_var']


/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


BX =  1.0*bmi_slope+1.0*lab_var
df columns: Index(['classification', 'race', 'sex'], dtype='object')
451 controls, 549 cases
pf: ['bmi_slope', 'lab_median', 'age_max']


/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/max/.conda/envs/epistasis/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


BX =  1.0*bmi_slope+1.0*lab_median+1.0*age_max
df columns: Index(['classification', 'race', 'sex'], dtype='object')
502 controls, 498 cases


In [43]:
print('running')
import pandas as pd                                                                  
import numpy as np                                                                   
from sklearn.metrics import accuracy_score                                               
from feat import Feat                                                                
                                                                                     
from glob import glob                                                                
problem = []                                                                         
solution = []                                                                        
models = []   
print(classification_equations[0])
seeds = [5990,9625,3620,13368,2882,3570,1011,7125]
for i,dataset in enumerate([d for d in glob('data/d_lr*.csv') if 'long' not in d]):
    curr_name = dataset[27:-4]
    print(curr_name)
    print(classification_equations[0].get(curr_name))
   

running
{'bmi_slope': '1.0*bmi_slope', 'lab_max': '1.0*lab_max', 'lab_min': '1.0*lab_min', 'lab_var': '1.0*lab_var', 'bmi_slope-lab_max': '1.0*bmi_slope+1.0*lab_max', 'bmi_slope-lab_slope': '1.0*bmi_slope+1.0*lab_slope', 'bmi_slope-lab_var': '1.0*bmi_slope+1.0*lab_var', 'bmi_slope-lab_median-age_max': '1.0*bmi_slope+1.0*lab_median+1.0*age_max'}
bmi_slope-lab_var
1.0*bmi_slope+1.0*lab_var
lab_max
1.0*lab_max
bmi_slope
1.0*bmi_slope
bmi_slope-lab_slope
1.0*bmi_slope+1.0*lab_slope
bmi_slope-lab_max
1.0*bmi_slope+1.0*lab_max
lab_var
1.0*lab_var
lab_min
1.0*lab_min
bmi_slope-lab_median-age_max
1.0*bmi_slope+1.0*lab_median+1.0*age_max


In [44]:
import pandas as pd                                                                  
import numpy as np                                                                   
from sklearn.metrics import accuracy_score                                               
from feat import Feat                                                                
                                                                                     
from glob import glob                                                                
problem = []                                                                         
solution = []                                                                        
models = []
equations = []

table_dicts = []

seeds = [5990,9625,3620,13368,2882,3570,1011,7125]
for i,dataset in enumerate([d for d in glob('data/d_lr*.csv') if 'long' not in d]):
    
    table_dicts.append(dict())
    
    curr_name = dataset[27:-4]
    print(curr_name)

    print('problem:',dataset)
    problem.append(dataset.split('example_patients_')[-1].split('.csv')[0])          
    df = pd.read_csv(dataset)                                                        
    ID = df['id']                                                                    
    df.drop('id',axis=1,inplace=True)                                                    
    df_X = df.drop(['classification'],axis=1)                                        
    feature_names = df_X.columns                                                     
    X = df_X.values                                                                  
    y = df['classification'].values                                                      
    zfile = dataset.split('.csv')[0] + '_long.csv'                                   
    Z = pd.read_csv(zfile)                                                           
                                                                                     
#     random_state = np.random.randint(2**14) 
    random_state = seeds[i]
    print('seed:',random_state)                                                      
    clf = Feat(max_depth=3, #force simplicity                                                              
               max_dim = 3,                                                          
               obj='fitness,size',   #size can change to complexity                                                   
               #gens = 1,                                                             
               #pop_size = 1,                                                           
               gens = 50,                                                           
               pop_size = 50,                                                          
               max_stall = 50,        #num of trees to test w/ that don't have better accuracy                                               
               stagewise_xo = True,                                                      
               scorer='log',                                                         
               verbosity=2,                                                              
               shuffle=True,                                                             
               ml='LR',                                                                  
               fb=0.1, #up the feedback to see what happens                                                             
               use_batch=False,                                                       
               batch_size=100,                                                       
               classification=True,                                                      
               feature_names = ','.join(feature_names),                              
               functions="+,-,*,/,split,split_c,"            
                         "mean,median,max,min,variance,skew,kurtosis,slope,count",       
                          # "slope",                                                 
               backprop=False,                                                           
               iters=3, #up the iterations to 3 to see if that changes anything                                                                  
               #random_state=random_state,                                                
               n_threads=8,                                                         
               split=0.5)                                                           
                                                                                     
    clf.verbosity = 2                                                                    
    clf.fit(X,y,zfile,ID.values)                                                     
    models.append(clf)                                                               
    solution.append(clf.get_representation())  
    equations.append(classification_equations[0].get(curr_name))
    print('solution archive:\n','\n'.join(models[-1].get_archive().split('\n')[:4]))
    print('accuracy score:',accuracy_score(y,clf.predict(X,zfile,np.arange(len(X)))))
    print('Classifier Equation: ' + str((classification_equations[0].get(curr_name))))
    
    table_dicts[i]['Equation'] = str((classification_equations[0].get(curr_name)))
    table_dicts[i]['Predictors'] = curr_name.split('-')
    table_dicts[i]['Solution Archive'] = '\n'.join(models[-1].get_archive().split('\n')[:4])
    table_dicts[i]['FEAT Best'] = clf.get_representation()
    #table_dicts[i]['FEAT Best Model'] = model.get_model()
    table_dicts[i]['FEAT Best Model'] = clf.get_model()
    table_dicts[i]['FEAT Best Accuracy'] = accuracy_score(y,clf.predict(X,zfile,np.arange(len(X))))
    
                                                                                     
for prob,soln,model,eqn in zip(problem,solution,models,equations):                                              
    print('////////////////////////////////////////')
    print('problem:',prob)                                                           
    print('solution rep:',soln)             
    print('solution model:\n',model.get_model())             
    print('solution archive:\n','\n'.join(model.get_archive().split('\n')[:4]))
    print('classifier equation: ' + str(eqn))
    print('////////////////////////////////////////')

bmi_slope-lab_var
problem: data/d_lr_example_patients_bmi_slope-lab_var.csv
seed: 5990
update_stats
exiting update_stats
solution archive:
 complexity	fitness	fitness_v	eqn
8	0.433569	1.144698	[slope(z_bmi)]
10	0.386523	1.298846	[slope(z_bmi)][max(z_lab)]
12	0.181120	2.096683	[slope(z_bmi)][max(z_lab)][min(z_lab)]
accuracy score: 0.754
Classifier Equation: 1.0*bmi_slope+1.0*lab_var
lab_max
problem: data/d_lr_example_patients_lab_max.csv
seed: 9625
update_stats
exiting update_stats
solution archive:
 complexity	fitness	fitness_v	eqn
6	0.009248	2.332071	[(max(z_lab)<1.142489)]
8	0.007745	2.561151	[max(z_lab)][(max(z_lab)<1.142489)]
8	0.578595	0.847721	[slope(z_lab)]
accuracy score: 0.738
Classifier Equation: 1.0*lab_max
bmi_slope
problem: data/d_lr_example_patients_bmi_slope.csv
seed: 3620
update_stats
exiting update_stats
solution archive:
 complexity	fitness	fitness_v	eqn
8	0.300913	2.456221	[slope(z_bmi)]
18	0.009375	2.337244	[(slope(z_bmi)<0.000038)]
26	0.008152	2.568390	[slope(z_bmi

In [45]:
df = pd.DataFrame(table_dicts)
print(df)
df.to_csv("simulation_results_repeated_trials_10.csv")

                                   Equation                  FEAT Best  \
0                 1.0*bmi_slope+1.0*lab_var             [slope(z_bmi)]   
1                               1.0*lab_max             [slope(z_lab)]   
2                             1.0*bmi_slope  [(slope(z_bmi)<0.000038)]   
3               1.0*bmi_slope+1.0*lab_slope             [slope(z_lab)]   
4                 1.0*bmi_slope+1.0*lab_max               [max(z_lab)]   
5                               1.0*lab_var               [max(z_lab)]   
6                               1.0*lab_min               [min(z_lab)]   
7  1.0*bmi_slope+1.0*lab_median+1.0*age_max               [min(z_age)]   

   FEAT Best Accuracy                                    FEAT Best Model  \
0               0.754      Feature\t\tWeight\nslope(z_bmi)\t\t2.167179\n   
1               0.738      Feature\t\tWeight\nslope(z_lab)\t\t1.255931\n   
2               0.999  Feature\t\tWeight\n(slope(z_bmi)<0.000034)\t\t...   
3               0.761      Fe